# Pydpf Basics Tutorial
This tutorial gives a basic overview of the differentiable particle filtering package, pydpf. The package is intended to implement the majority of differentiable filters currently available in the literature and provide a convenient API for training them. It is part of a larger project to develop a benchmarking suite for DPFs. This tutorial assumes good knowledge of pytorch.

## Design principles
1. Extensibility
    It should be as easy as possible for a user to define and train a filter to their needs without the package getting in the way. It should generally be possible to extend the functionality we provide by passing values to package functions, rather than subclassing package classes.
2. Pytorch semantics
    This module integrates with pytorch. Typical torch semantics are preserved as much as possible.
3. Learning functions not models.
    Conceptually it is best to think of modules written for pydpf as learning an algorithm to achieve the given inference goal, not a specific state space model. This gives the user flexibility in structuring their model as best suited to their problem rather than having to conform to the package's syntax. Most particle filtering algorithms are implementable as 'models' that can be passed to our base SMC algorithms, rather than needing to design them from scratch. Conversely, this puts the responsibility of designing well-structured and correct filtering algorithms on the user.

## Overarching design patterns

### The pydpf.Module class
pydpf has a custom Module class that extends torch.nn.Module. pydpf.Module subclasses can optionally contain an update() method that should be used to calculate derived quantities from, or constrain parameters after they have changed. We recommend that all custom modules defined for use with pydpf subclass pydpf.Module. update() is called recursively on submodules, for this reason it is safe for pydf.Module classes to have torch.nn.Module submodules but not vice versa. So, there is no issue in using built in torch modules. Consequently, it is not safe to directly set attributes of a Module, one should provide a safe set_attribute method for Module attributes. The exception to this rule is when using torch's (somewhat arcane) optimizers which update the parameters inplace. One should then make sure to call .update() on all root Modules following an Optimizer step.

### Dataformat
@Ben this is subject to change if required. The data-format currently required by the pydpf data loading faculties is a folder of .csv files where each file contains a trajectory. Each file should have two columns, labelled state and observations containing vectors in the format '[state_1, state_2, ..., state_n]' (quotes included). Other columns are permitted but will be ignored.  

### Order of dimensions
The order of dimensions shall be: time, batch, particle, state/observation-dimension. Frequently one or more of these dimensions shall not be present but they should not change relative order.

In [1]:
#imports
import pydpf
import torch
from typing import Tuple
from pydpf import Module, constrained_parameter
from torch.utils.data import DataLoader
import numpy as np

#set device, defaults to cuda if it is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#set data path
data_path = "./data"

## Defining the true Model
This tutorial takes the user through a simple two-dimensional Linear Gaussian example.
At time = 0 the state is drawn from a Gaussian, at subsequent time-steps it is drawn from a Gaussian Markov kernel with linear dependence on the previous state. The observations are drawn from a different linear Gaussian forward kernel. The linear transforms and covariance matrices are constant for all time steps. 

In [2]:
#Define the true parameters
true_prior_mean = torch.zeros(size = (2,), dtype = torch.float32, device = device)
true_prior_cholesky_cov = torch.eye(2, dtype = torch.float32, device = device)
true_dynamic_scaling = torch.eye(n=2, dtype = torch.float32, device = device) * 0.9
true_dynamic_offset = torch.tensor([0.2, 0.1], dtype = torch.float32, device = device)
true_dynamic_cholesky_cov = torch.tensor([[0.1, 0], [0.03, 0.1]], dtype = torch.float32, device = device)
true_observation_scaling = torch.eye(n = 2, dtype = torch.float32, device = device)
true_observation_offset = torch.tensor([0, 0], dtype = torch.float32, device = device)
true_observation_cholesky_cov = torch.eye(2, dtype = torch.float32, device = device)*0.05

## The distributions module
We include a distributions module to implement common distributions (and conditional distributions) with a convenient API. Like the torch.distributions package it is not intended for the user to subclass the Distribution object, if they wish to implement their own it will always be easier to manually define sampling and density evaluation methods. Distribution objects implement a sample() and log_density() method, the specific arguments of which depend on the specific distribution. Distributions are pydpf.Module subclasses so may be assigned parameters. Distributions give three built in options for the gradient estimator to use for sampling, 'reparameterisation' reparameterisation trick, 'score' score-based/REINFORCE, and 'none' detach gradients. Although, attaching a score-based gradient estimator to the sample requires doing so in linear-space, whereas attaching it to the importance weights can be done in log-space so may be more stable in practice.

In [3]:
true_prior = pydpf.distributions.MultivariateGaussian(gradient_estimator = 'none', generator = None, mean = true_prior_mean, cholesky_covariance = true_prior_cholesky_cov)

true_dynamic = pydpf.distributions.LinearGaussian(gradient_estimator='none', generator= None, weight =  true_dynamic_scaling, bias = true_dynamic_offset, cholesky_covariance = true_dynamic_cholesky_cov)

true_observation = pydpf.distributions.LinearGaussian(gradient_estimator='none', generator= None, weight =  true_observation_scaling, bias = true_observation_offset, cholesky_covariance = true_observation_cholesky_cov)

def dynamic_sampler(prev_state, t):
    return true_dynamic.sample(prev_state, None)

def observation_sampler(state, t):
    return true_observation.sample(state, None)

## Saving simulated data
We provide a method to generate a set of trajectories and save them to folder as .csv files as required by our format. The simulate_to_folder method takes methods to sample from the prior and the dynamic and observation kernels.

In this case the sampling functions are the .sample() methods of the distributions defined above.

Note: sometimes running joblib processes in a jupyter notebook crashes, just rerun the cell should this happen.

In [4]:
pydpf.simulate_to_folder(data_path, true_prior.sample, dynamic_sampler, observation_sampler, time_extent=50, n_trajectories=1000, batch_size=30, device=device, processes=-1)

Warning - folder already exists at ./data, continuing could overwrite its data


KeyboardInterrupt: Interrupted by user

## Initialising model parameters
In this example we will only learn the dynamic and observation models, not the prior. We randomly initialise the parameters as torch.nn.Parameters and define new distributions to hold them.

In [ ]:
initialisation_generator = torch.Generator(device = device).manual_seed(0)
learned_dynamic_scaling = torch.nn.Parameter(torch.rand(size = (2,2), dtype = torch.float32, device = device, generator=initialisation_generator) * 2 - 1, requires_grad = True)
learned_dynamic_offset = torch.nn.Parameter(torch.rand(size = (2,), dtype = torch.float32, device = device, generator=initialisation_generator) * 2 - 1, requires_grad = True, )
learned_dynamic_cholesky_cov = torch.nn.Parameter(torch.rand(size = (2,2), dtype = torch.float32, device = device, generator=initialisation_generator), requires_grad = True)
learned_observation_scaling = torch.nn.Parameter(torch.rand(size = (2,2), dtype = torch.float32, device = device, generator=initialisation_generator) * 2 - 1, requires_grad = True)
learned_observation_offset = torch.nn.Parameter(torch.rand(size = (2,), dtype = torch.float32, device = device, generator=initialisation_generator) * 2 - 1, requires_grad = True)
learned_observation_cholesky_cov = torch.nn.Parameter(torch.rand(size = (2,2), dtype = torch.float32, device = device, generator=initialisation_generator), requires_grad = True)

prior_rng = torch.Generator(device = device)
prior_rng.manual_seed(0)
dynamic_rng = torch.Generator(device = device)
dynamic_rng.manual_seed(1)
observation_rng = torch.Generator(device = device)
observation_rng.manual_seed(2)


filtering_prior = true_prior


filtering_dynamic = pydpf.distributions.LinearGaussian(gradient_estimator = 'reparameterisation', generator = dynamic_rng, weight = learned_dynamic_scaling, bias = learned_dynamic_offset, cholesky_covariance = learned_dynamic_cholesky_cov)

filtering_observation = pydpf.distributions.LinearGaussian(gradient_estimator = 'reparameterisation', generator = observation_rng, weight = learned_observation_scaling, bias = learned_observation_offset, cholesky_covariance = learned_observation_cholesky_cov)

## Defining the DPF
ParticleFilter is our implementation of a (not necessarily differentiable) particle filter. The particle filter is defined in terms of three functions. A function to importance sample posterior at time zero; takes the number of particles and the observation at time zero. A function to importance sample from the posterior at subsequent time-steps; takes the previous states, the previous weights, the observations at the current time and the current time. And a resampling algorithm; takes the particles and weights and returns the resampled particles, weights and another tensor (used to pass other information for outputting, most often the resampled indices).

In our example we demonstrate the usage of the .update() function by using it to constrain the dynamic matrix's spectral index. For this example we employ the non-differentiable multinomial resampling.

In [ ]:
class PriorSampler(Module):
    def __init__(self):
        super().__init__()
        self.prior_model = filtering_prior
        self.observation_model = filtering_observation

    def forward(self, n_particles: int, data: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        new_state = self.prior_model.sample((data.size(0), n_particles))
        new_weights = self.observation_model.log_density(data.unsqueeze(1), new_state)
        return new_state, new_weights
    
    

class FilteringSampler(Module):
    def __init__(self):
        super().__init__()
        self.dynamic_model = filtering_dynamic
        self.observation_model = filtering_observation
        
    def forward(self, state: torch.Tensor, weights: torch.Tensor, data: torch.Tensor, time: int) -> Tuple[torch.Tensor, torch.Tensor]:
        new_state = self.dynamic_model.sample(state)
        new_weights = self.observation_model.log_density(data.unsqueeze(1), new_state) + weights
        return new_state, new_weights
        
    @constrained_parameter
    def constrained_params(self):
        dynamic_eigvals = torch.linalg.eigvals(self.dynamic_model.weight)
        dynamic_spectral_radius = torch.max(torch.abs(dynamic_eigvals))
        if dynamic_spectral_radius > 1:
            return self.dynamic_model.weight, self.dynamic_model.weight/dynamic_spectral_radius
        return self.dynamic_model.weight, self.dynamic_model.weight

resampling_generator = torch.Generator(device=device).manual_seed(6)

dpf = pydpf.ParticleFilter(PriorSampler(), pydpf.multinomial(resampling_generator), FilteringSampler())

## Loading data
Data loading closely follows pytorch syntax, we have defined a custom dataset to load data from a folder of formatted .csv files. The dataset can then be treated as a regular pytorch dataset. With the exception that one should pass its .collate() routine to the DataLoader collate_fn argument.

In [ ]:
dataset_generator = torch.Generator(device='cpu').manual_seed(3)
train_loader_generator = torch.Generator(device='cpu').manual_seed(4)
validation_loader_generator = torch.Generator(device='cpu').manual_seed(5)
dataset = pydpf.StateSpaceDataset(data_path, device=device)
train_set, validation_set = torch.utils.data.random_split(dataset, [500, 500], generator=dataset_generator)
train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True, generator=train_loader_generator, collate_fn=dataset.collate)
validation_dataloader = DataLoader(validation_set, batch_size=128, shuffle=True, generator=validation_loader_generator, collate_fn=dataset.collate)

In [ ]:
#Hyper-parameters
epochs = 20
lr = 0.001
n_particles_train = 100
n_particles_validation = 1000

## Training loop
It is often crucial to have control over the training loop to debug and tune models so, like base pytorch, we leave their implementation to the user. The simple algorithm in this example is not very good, it is only for the point of demonstration, much better losses for this example are possible.

In [ ]:
opt = torch.optim.SGD(dpf.parameters(), lr=lr)

for epoch in range(epochs):
    train_loss = []
    for state, observation in train_dataloader:
        dpf.update()
        opt.zero_grad()
        filtering_means = dpf(observation, n_particles_train, observation.size(0)-1, pydpf.filtering_mean())
        loss = pydpf.MSE_loss(filtering_means, state)
        loss.backward()
        train_loss.append(loss.item())
        opt.step()
    train_loss = np.mean(np.array(train_loss))
    dpf.update()
    with torch.inference_mode():
        for state, observation in validation_dataloader:
            filtering_means = dpf(observation, n_particles_validation, observation.size(0) - 1, pydpf.filtering_mean())
            loss = pydpf.MSE_loss(filtering_means, state)
            validation_loss = loss.item()
    print('                                                                                                    ', end='\r')
    print(f'epoch {epoch+1}/{epochs}, train loss: {train_loss}, validation loss: {validation_loss}', end='\r')
    
print('')